In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import time 
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import math

In [ ]:

#loading the csv files

lidar=pd.read_csv("Flight02_pointcloud.csv")

gps=pd.read_csv("Flight02_gps.csv")

#converting the data of required rows to list item
m = lidar["x"]
m= list(m)

c=gps["timestamp"]
c=list(c)

t = lidar['timestamp']
t=list(t)

t1=list(set(t))
#print(t)

In [ ]:

angle=list(lidar["theta"])
utc=list(gps["time_utc_usec"])


#loading the intensity coloumn and using it as one of the thresholds
#finding mean of the intensity to find the middle value. Greater than mean value is approximated as muddy road,
#less than mean value is approximated as grey flat road
intensity=list(lidar["intensity"])
mean_i=np.mean(np.array(intensity))

#print(mean_i)



v=[]
a=[]

count=0
i=0

#if the difference between consecutive x axis lidar values is very high , then we predict that a pothole exists

while(i<len(m)-1):
    #if ((( m[i]*(abs(180-angle[i]))-(m[i+1]*(abs(180-angle[i+1]))) ))>0.5):
    if (((m[i+1]-(m[i])))>2.7 and intensity[i]>mean_i):
        count=count+1
        
        #calculate the volume of the pothole approximately in m3
        radius = abs(m[i+3]-m[i+1])
        volume=2*radius*3.14/3
        
        v.append(volume)
        
        a.append(t[i])
        i=i+77
        """
        if i+90000 < len(m)-1:
            i=i+90000
        else:
            i=i+10000
        #print(t[i],m[i],m[i+1])
        """
        
    else:
        i=i+1
        
        
print(count,"places to fix road")



#Here we take the timestamp values only when the pothole is predicted and compare these timetsamp values 
#with the values in gps csv file. The corresponding longitude and lattitude value for the particular
#timestamp is found and plotted to the map

lat=[]
long=[]
tm=[]

lt=list(gps["lat"])
lo=list(gps["lon"])
tm1=list(gps["time_utc_usec"])

for i in range(0,len(a)):
    for j in range(0,len(c)):
        #print((a[i]/1000000000-c[j]/1000000) , a[i]/1000000000,c[j]/1000000)
        if (math.floor(a[i]/1000000000-c[j]/1000000)==0):
            lat.append(lt[j]/10000000)
            long.append(lo[j]/10000000)
            tm.append(tm1[j])
#            print(lt[j]/10000000,lo[j]/10000000)


print(lat,long,v)






In [ ]:
import gmplot
import csv
with open('lat.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow("lat")
    for val in lat:
        writer.writerow([val])
        
with open('long.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow("long")
    for val in long:
        writer.writerow([val])
        
with open('volume.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow("vol")
    for val in v:
        writer.writerow([val])
        
with open('tm.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow("timestamp(utc)")
    for val in tm1:
        writer.writerow([val])



In [ ]:
gmap = gmplot.GoogleMapPlotter(lat[0], long[0],16)

gmap.plot(lat, long,'cornflowerblue', edge_width=10)
gmap.scatter(lat, long, '#ff0000', size=1, marker=False)

#gmap.plot(list(np.array(lt)/10000000), list(np.array(lo)/10000000),'cornflowerblue', edge_width=10)
#gmap.scatter(list(np.array(lt)/10000000), list(np.array(lo)/10000000), '#3B0B39', size=1, marker=False)
#gmap.scatter(marker_lats, marker_lngs, 'k', marker=True)
#gmap.heatmap(heat_lats, heat_lngs)

gmap.draw("mymap.html")